In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import os

os.listdir('/kaggle/input/plantdisease/PlantVillage')
train_data_dir = '/kaggle/input/plantdisease/PlantVillage'

In [ ]:
# from keras.datasets import mnist,cifar10,fashion_mnist
import keras
import os
# ,shutil
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, Dropout, Activation
from keras.optimizers import Adam
import matplotlib.pyplot as plt

In [ ]:
# trdata = ImageDataGenerator()
# traindata = trdata.flow_from_directory(directory="/kaggle/input/dogs-cats-images/dataset/training_set",target_size=(224,224))#, batch_size=64)
# tsdata = ImageDataGenerator()
# testdata = tsdata.flow_from_directory(directory="/kaggle/input/dogs-cats-images/dataset/test_set", target_size=(224,224))#, batch_size=64)
# os.listdir('/kaggle/input/dogs-cats-images/dataset')

img_height = 224
img_width = 224
batch_size = 64

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
#     class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
#     class_mode='binary',
    subset='validation') # set as validation data

In [ ]:
##--##--##--##--##--##--##--##--##--##--##--# """VGG16""" #--##--##--##--##--##--##--##--##--##--##--##
# model = Sequential()# (224,224,3)
# model.add(Conv2D(input_shape=(224,224,3),filters=32,kernel_size=(3,3),padding="same", activation="relu"))
# model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
# model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
# model.add(Flatten())
# model.add(Dense(units=1024,activation="relu"))
# model.add(Dense(units=1024,activation="relu"))
# model.add(Dense(units=15, activation="softmax"))

##--##--##--##--##--##--##--##--##--##--##--# """MOBILENET""" #--##--##--##--##--##--##--##--##--##--##--##

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(15))
model.add(Activation('softmax'))

In [ ]:
opt = Adam(lr=0.001)

#compile model using accuracy to measure model performance
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
#train the model
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("/kaggle/working/mobilenet_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

# hist = model.fit_generator(steps_per_epoch=25,generator=traindata, validation_data=testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])
nb_epochs = 22

hist = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs,
    callbacks=[checkpoint,early]
)

In [1]:
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

NameError: name 'plt' is not defined

In [2]:
filename = '/kaggle/working/mobilenet_1.h5'
from IPython.display import FileLink
FileLink(filename)

/kaggle/working/mobilenet_1.h5

In [27]:
# import smtplib 
# from email.mime.multipart import MIMEMultipart 
# from email.mime.text import MIMEText 
# from email.mime.base import MIMEBase 
# from email import encoders 
   
# fromaddr = "abhijitacharya793@gmail.com"
# toaddr = "abhijitacharya793@gmail.com"
   
# # instance of MIMEMultipart 
# msg = MIMEMultipart() 
  
# # storing the senders email address   
# msg['From'] = fromaddr 
  
# # storing the receivers email address  
# msg['To'] = toaddr 
  
# # storing the subject  
# msg['Subject'] = "Mobilenet H5 model"
  
# # string to store the body of the mail 
# body = "Nothing"
  
# # attach the body with the msg instance 
# msg.attach(MIMEText(body, 'plain')) 
  
# # open the file to be sent  
# filename = '/kaggle/working/mobilenet_1.h5'
# attachment = open(filename, "rb") 
  
# # instance of MIMEBase and named as p 
# p = MIMEBase('application', 'octet-stream') 
  
# # To change the payload into encoded form 
# p.set_payload((attachment).read()) 
  
# # encode into base64 
# encoders.encode_base64(p) 
   
# p.add_header('Content-Disposition', "attachment; filename= %s" % filename) 
  
# # attach the instance 'p' to instance 'msg' 
# msg.attach(p) 
  
# # creates SMTP session 
# s = smtplib.SMTP('smtp.gmail.com', 587) 
  
# # start TLS for security 
# s.starttls() 
  
# # Authentication 
# s.login(fromaddr, "Password_of_the_sender") 
  
# # Converts the Multipart msg into a string 
# text = msg.as_string() 
  
# # sending the mail 
# s.sendmail(fromaddr, toaddr, text) 
  
# # terminating the session 
# s.quit() 